In [ ]:
# !pip install stanza
# !pip install ollama
# !pip install vertexai
# !pip install -t lib google-auth google-auth-httplib2 google-api-python-client --upgrade
# !pip install pandas_gbq
# !pip install python-dotenv

In [1]:
import pandas as pd
import os
import re
import stanza
import random
import logging
import pymorphy3
import json
import shutil

from tqdm import tqdm

tqdm.pandas()

import numpy as np
import spacy
# import tqdm
# from langchain_text_splitters import RecursiveCharacterTextSplitter

# nlp = spacy.load("uk_ner_web_trf_13class")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.4 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/usr/local/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/usr/local/Cellar/python@3.10/3.10.16/Frameworks/Python.framework/Versions/3.10/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/Users/linndfors/study/diploma/new_venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/lin

## LLM

In [ ]:
import openai
from openai import OpenAI

import os
from dotenv import load_dotenv

load_dotenv(".env")

openai.api_key = os.environ.get("OPENAI_API_KEY")
first_names_with_sex_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/data/util/first_names_frequency_popular.csv")

In [3]:
def fill_prompt_with_names(female_names, male_names):
    updated_prompt = f"""
Ви - експерт з обробки природної мови. Ваше завдання - змінити задане речення зі зміною статі, використовуючи словник анотацій у форматі {{"слово": "його сутність(label) з NER"}}. Дотримуйтесь таких правил:
    1. Якщо зустрічається слово чоловічого роду з класу "JOB", змініть його на жіночий аналог, і навпаки для жіночого роду. 
       **Виняток**: якщо ця сутність класу "JOB" належить до **спільного роду** (наприклад, "голова", "суддя"), такі слова, а також усі пов'язані з ними залежні слова (імена, прикметники, дієслова тощо), **залишаються без змін**.
    2. Якщо "JOB" було змінене, то змінити потрібно і сутність "PERS".
    3. Якщо слова не відносятся до сутностей в анотаціях, то їх змінювати непотрібно.
    Знайди в сутності "PERS" імʼя і:
    а) якщо імʼя належить до чоловічого роду то зміни на жіноче, обране лише з наступних запропонованих варіантів: {female_names}.
    б) якщо імʼя належить до жіночого роду то зміни на чоловіче, обране лише з наступних запропонованих варіантів: {male_names}.
    Не можна використосувати інші імена, окрім запропонованих.
    4. Зберігайте відмінки слів: якщо слово було, наприклад, у родовому відмінку ("режисера"), то його аналог має теж бути в родовому відмінку ("режисерки").
    5. Зберігайте форму слова за числом: множина залишається множиною ("власники" → "власниці"), однина - одниною.
    6. Забезпечте граматичну коректність речення, узгоджуючи змінені слова з контекстом.
    7. Залиши всі інші знаки, пунктуації, символи, включаючи символи "<" і ">", незмінними.
    8. Як результат виведи змінене речення без жодних додаткових пояснень.

    Приклад:
        Input: Дмитро Петренко став новим адміністратором за наказом заступниці директора Тетяни Сірої, яка належить до прихильників радикальних методів. {{"Дмитро Петренко": "PERS", "Тетяни Сірої": "PERS", "адміністратором": "JOB", "заступниці": "JOB", "директора": "JOB"}}
        Output: Марина Петренко стала новою адміністраторкою за наказом заступника директорки Володимира Сірого, який належить до прихильників радикальних методів.

        Input: Сестри художника Бориса Мартинюка були відомими співачками, тому у Бориса не було часу проводити час з подругами. {{"художника": "JOB", "Бориса Мартинюка": "PERS", "співачками": "JOB", "Бориса": "PERS"}}
        Output: Брати художниці Олени Мартинюк були відомими співаками, тому у Олени не було часу проводити час з подругами.
        
        Input: Суддя Марина зʼявилась запізно <.{{"Суддя": "JOB", "Марина": "PERS"}}
        Otput: Суддя Марина зʼявилась запізно <.
"""
    return updated_prompt

In [4]:
def generate_random_name(df, n, sex=None):
    if sex:
        df = df[df['sex'] == sex]
    
    names = df['name']
    frequencies = df['freq']
    
    return list(np.random.choice(names, size=n, p=frequencies/np.sum(frequencies)))


def change_gender(sentence):
    male_names = generate_random_name(first_names_with_sex_df, 3, 'M')
    female_names = generate_random_name(first_names_with_sex_df, 3, 'F')
    print("male names:", male_names)
    print("female names:", female_names)

    gender_swap_prompt = fill_prompt_with_names(female_names, male_names)

    messages = [
        {"role": "system", "content": gender_swap_prompt},
        {"role": "user", "content": sentence},
    ]

    response = openai.chat.completions.create(
        model="gpt-4o-mini", messages=messages, temperature=0.8, max_tokens=250, top_p=1
    )

    response_content = response.choices[0].message.content
    return response_content

# Prepare data

In [ ]:
def find_ann_files(directory):
    ann_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".ann"):
                file_path = os.path.join(root, file)
                ann_files.append(file_path)

    return ann_files

def read_ann_files(directory):
    ann_files = find_ann_files(directory)
    job_title_sentences = []

    for ann_file in ann_files:
        with open(ann_file, "r", encoding="utf-8") as ann_f:
            for ann_l in ann_f:
                ann_row = ann_l.split("\t")

                try:
                    if ann_row[1] == "JOB":
                        job_title_sentences.append(ann_row)
                except Exception as e:
                    continue

    return job_title_sentences

def find_txt_files(directory):
    txt_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                file_path = os.path.join(root, file)
                txt_files.append(file_path)

    return txt_files

def read_txt_files(directory):
    txt_files = find_txt_files(directory)
    text = []

    for txt_file in txt_files:
        with open(txt_file, "r", encoding="utf-8") as txt_f:
            text.append(txt_f.read())

    return text

text_ng = read_txt_files("/Users/linndfors/study/diploma/ner_for_fem/data/v2.0/data/ng")
text_bruk = read_txt_files("/Users/linndfors/study/diploma/ner_for_fem/data/v2.0/data/bruk")

In [ ]:
def parse_ann_file(ann_file):
    entities = []

    with open(ann_file, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split("\t")
            if len(parts) < 3:
                continue
            label = parts[1]
            start = parts[2]
            end = parts[3]
            text = parts[4]

            entities.append({"text": text, "start": start, "end": end, "label": label})

    return entities


def extract_entities(txt_file, ann_file):
    labeled_sentences_list = []
    sentences = []

    with open(txt_file, "r", encoding="utf-8") as f:
        sentence_lines = f.readlines()

    entities = parse_ann_file(ann_file)

    for sentence in sentence_lines:

        labeled_dict = {}
        job = False
        sentence = sentence.strip()

        for ent in entities:
            if re.search(rf"\b{re.escape(ent['text'])}\b", sentence):
                labeled_dict[ent["text"]] = ent["label"]
                if ent["label"] == "JOB":
                    job = True

        if job:
            labeled_sentences_list.append(labeled_dict)
            sentences.append(sentence)

    return labeled_sentences_list, sentences

In [ ]:
def read_data(file_path):
    data_dict = {}
    current_label = None

    with open(file_path, "r") as file:
        for line in file:
            line = line.strip()

            if line in ["DEV", "TEST"]:
                current_label = line
                data_dict[current_label] = []
            elif current_label:
                if line:
                    data_dict[current_label].append(line)

    return data_dict

file_path = "/Users/linndfors/study/diploma/ner_for_fem/data/v2.0/data/dev-test-split.txt"
data_dict = read_data(file_path)

In [6]:
def find_files(directory):
    dev_list = []
    test_list = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(".txt"):
                if file.split(".")[0] in list(data_dict["DEV"]):
                    file_path = os.path.join(root, file)
                    dev_list.append(file_path)
                elif file.split(".")[0] in list(data_dict["TEST"]):
                    file_path = os.path.join(root, file)
                    test_list.append(file_path)

    return dev_list, test_list


def read_files(txt_files):
    text = ""
    for txt_file in txt_files:
        with open(txt_file, "r", encoding="utf-8") as txt_f:
            text += txt_f.read()

    return text

In [ ]:
bruk_dev, bruk_test = find_files("/Users/linndfors/study/diploma/ner_for_fem/data/v2.0/data/bruk")
ng_dev, ng_test = find_files("/Users/linndfors/study/diploma/ner_for_fem/data/v2.0/data/ng")

dev_files = bruk_dev + ng_dev
test_files = bruk_test + ng_test

ng_files = ng_dev + ng_test
bruk_files = bruk_dev + bruk_test 

dev_files_text = read_files(dev_files)
test_files_text = read_files(test_files)

dev_ng_text = read_files(ng_dev)
test_ng_text = read_files(ng_test)

dev_bruk_text = read_files(bruk_dev)
test_bruk_text = read_files(bruk_test)

In [ ]:
def get_sentences_with_job_titles(txt_file_names):

    sentence_with_jobs = {}

    for txt_file in txt_file_names:
        ann_file = txt_file.replace(".txt", ".ann")
        labeled_sentences_list, sentences = extract_entities(txt_file, ann_file)
        for i in range(len(sentences)):
            sentence_with_jobs[sentences[i]] = labeled_sentences_list[i]

    return sentence_with_jobs

test_sentences_with_jobs = get_sentences_with_job_titles(test_files)
dev_sentences_with_jobs = get_sentences_with_job_titles(dev_files)

In [12]:
ng_jobs = get_sentences_with_job_titles(ng_files)
bruk_jobs = get_sentences_with_job_titles(bruk_files)

In [11]:
len(ng_jobs)

1020

In [13]:
len(bruk_jobs)

522

In [26]:
list_of_jobs_ng = []

for x, val in ng_jobs.items():
    for ent, lab in val.items():
        if lab == "PERS":
            list_of_jobs_ng.append(ent)

In [22]:
list_of_jobs_bruk = []

for x, val in bruk_jobs.items():
    for ent, lab in val.items():
        if lab == "PERS":
            list_of_jobs_bruk.append(ent)

In [31]:
with open('ng_pers_orig_only_jobs_sents.txt', 'w', encoding='utf-8') as f:
    for item in list_of_jobs_ng:
        f.write(f"{item}\n")

In [ ]:
# with open("/Users/linndfors/study/diploma/ner_for_fem/JSON_files/test_sentences_with_jobs.json", "w", encoding="utf-8") as f:
#     json.dump(test_sentences_with_jobs, f, ensure_ascii=False, indent=4)

# with open("/Users/linndfors/study/diploma/ner_for_fem/JSON_files/dev_sentences_with_jobs.json", "w", encoding="utf-8") as f:
#     json.dump(dev_sentences_with_jobs, f, ensure_ascii=False, indent=4)

# Run Gender-swapping

In [ ]:
gender_swaped_pairs_dev = {}

for sentence, ann_dict in tqdm(
    dev_sentences_with_jobs.items(), desc="Processing sentences"
):
    try:
        filtered_ann_dict = {key: value for key, value in ann_dict.items() if value in {'PERS', 'JOB'}}
        dev_raw_and_annotated = sentence + "\n" + str(filtered_ann_dict)
        # print(test_raw_and_annotated)
        output = change_gender(dev_raw_and_annotated)
        print(dev_raw_and_annotated)
        print(output)
        print("-------------------------------------")
        gender_swaped_pairs_dev[sentence] = output
    except Exception as e:
        print("error:", e)

In [79]:
gender_swaped_df = pd.DataFrame(
    gender_swaped_pairs_dev.items(), 
    columns=["original", "swapped"]
)

In [80]:
gender_swaped_df.head()

,original,swapped
0,Виставу за п'єсою російського класика Льва Тол...,Виставу за п'єсою російського класика Льва Тол...
1,Віртуозні Лесь Задніпровський – князь Абрезков...,Віртуозні Ганна Задніпровська – княгиня Абрезк...
2,"Старша в Києві , одружена з військовим , чолов...","Старша в Києві , одружена з військовою , чолов..."
3,"Ні , не так - автор статті , позаштатний корес...","Ні , не так - авторка статті , позаштатна коре..."
4,І лектори в сірих піджаках із червоними парткв...,І лектори в сірих піджаках із червоними парткв...


### Creating dataset for annotation project

In [ ]:
df_name = "/Users/linndfors/study/diploma/ner_for_fem/annotation_project/sentences_for_annotation/dev_swapped_sentences_with_names_frequencies.csv"
gender_swaped_df.to_csv(df_name)

Add old annotation

In [ ]:
# read swapped sentences
import csv

orig_and_changed_dict = {}

with open("/Users/linndfors/study/diploma/ner_for_fem/annotation_project/sentences_for_annotation/dev_swapped_sentences_with_names_frequencies.csv", mode='r', encoding='utf-8') as file:
    reader = csv.reader(file)
    
    next(reader)
    for row in reader:
        key = row[1]
        value = row[2]
        orig_and_changed_dict[key] = value

with open("/Users/linndfors/study/diploma/ner_for_fem/JSON_files/test_sentences_with_jobs.json", "r", encoding="utf-8") as f:
    test_sentences_with_jobs = json.load(f)

with open("/Users/linndfors/study/diploma/ner_for_fem/JSON_files/dev_sentences_with_jobs.json", "r", encoding="utf-8") as f:
    dev_sentences_with_jobs = json.load(f)

def set_annotattions(original_sent):
    if original_sent in dev_sentences_with_jobs.keys():
        annotations = dev_sentences_with_jobs[original_sent] 
        filtered_annotations = {key: value for key, value in annotations.items() if value in {'PERS', 'JOB'}}
        return filtered_annotations
    return None

gender_swaped_df['old_Annotations'] = gender_swaped_df.apply(lambda x: set_annotattions(x['original']), axis=1)
gender_swaped_df.to_csv("/Users/linndfors/study/diploma/ner_for_fem/annotation_project/sentences_for_annotation/dev_swapped_with_ann_with_names_changed.csv", index=False)

# Annotation for .ann files reseting (for the Gender-balanced dataset)

In [2]:
dev_swapped_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/annotation_project/sentences_after_annotation/dev_swapped.csv")
test_swapped_df = pd.read_csv("/Users/linndfors/study/diploma/ner_for_fem/annotation_project/sentences_after_annotation/test_swapped.csv")

swapped_df = pd.concat([dev_swapped_df, test_swapped_df], ignore_index=True)

In [52]:
swapped_df.head()

,Оригінальне речення,Змінене речення,Анотації,Коректність речення,Виправлене речення,Помічник/ця
0,Виставу за п'єсою російського класика Льва Тол...,Виставу за п'єсою російського класика Льва Тол...,"{'Льва Толстого': 'PERS', 'режисера': 'JOB', '...",Містить помилки,Виставу за п'єсою російської класикині Марії Т...,Оля Н
1,Віртуозні Лесь Задніпровський – князь Абрезков...,Віртуозні Ганна Задніпровська – княгиня Абрезк...,"{'Лесь Задніпровський': 'PERS', 'князь': 'JOB'...",Містить помилки,Віртуозні Ганна Задніпровська – княгиня Абрезк...,NaN
2,"Старша в Києві , одружена з військовим , чолов...","Старша в Києві , одружена з військовою , чолов...",{'військовим': 'JOB'},Містить помилки,"Старша в Києві , одружена з військовою , жінка...",NaN
3,"Ні , не так - автор статті , позаштатний корес...","Ні , не так - авторка статті , позаштатна коре...",{'позаштатний кореспондент': 'JOB'},Правильне,NaN,NaN
4,І лектори в сірих піджаках із червоними парткв...,І лектори в сірих піджаках із червоними парткв...,{'священиків': 'JOB'},Правильне,NaN,NaN


In [53]:
swapped_df['gender-swapped'] = swapped_df.apply(lambda x: x['Виправлене речення'] if x['Виправлене речення'] and x['Коректність речення']=='Містить помилки' else x['Змінене речення'], axis=1)

In [54]:
swapped_df[(swapped_df['Коректність речення']=='Містить помилки') & (swapped_df['Виправлене речення'].isna())]

,Оригінальне речення,Змінене речення,Анотації,Коректність речення,Виправлене речення,Помічник/ця,gender-swapped


In [55]:
swapped_df.head()

,Оригінальне речення,Змінене речення,Анотації,Коректність речення,Виправлене речення,Помічник/ця,gender-swapped
0,Виставу за п'єсою російського класика Льва Тол...,Виставу за п'єсою російського класика Льва Тол...,"{'Льва Толстого': 'PERS', 'режисера': 'JOB', '...",Містить помилки,Виставу за п'єсою російської класикині Марії Т...,Оля Н,Виставу за п'єсою російської класикині Марії Т...
1,Віртуозні Лесь Задніпровський – князь Абрезков...,Віртуозні Ганна Задніпровська – княгиня Абрезк...,"{'Лесь Задніпровський': 'PERS', 'князь': 'JOB'...",Містить помилки,Віртуозні Ганна Задніпровська – княгиня Абрезк...,NaN,Віртуозні Ганна Задніпровська – княгиня Абрезк...
2,"Старша в Києві , одружена з військовим , чолов...","Старша в Києві , одружена з військовою , чолов...",{'військовим': 'JOB'},Містить помилки,"Старша в Києві , одружена з військовою , жінка...",NaN,"Старша в Києві , одружена з військовою , жінка..."
3,"Ні , не так - автор статті , позаштатний корес...","Ні , не так - авторка статті , позаштатна коре...",{'позаштатний кореспондент': 'JOB'},Правильне,NaN,NaN,"Ні , не так - авторка статті , позаштатна коре..."
4,І лектори в сірих піджаках із червоними парткв...,І лектори в сірих піджаках із червоними парткв...,{'священиків': 'JOB'},Правильне,NaN,NaN,І лектори в сірих піджаках із червоними парткв...


In [56]:
orig_and_changed_dict = dict(zip(swapped_df['Оригінальне речення'], swapped_df['gender-swapped']))

In [57]:
# change original text with the sentences we gender-swapped (sentences with JOB TITLEs)
all_files_text = dev_files_text + test_files_text
all_sent = all_files_text.split("\n")


print("Sentence to change:", len(all_sent))

Sentence to change: 24047


In [ ]:
def format_punctuation(text):
    text = re.sub(r'\s+([,()?!:])', r'\1', text)
    text = re.sub(r'\s+(\.)', r'\1', text)
    
    text = re.sub(r'(?<!\s)([,.()?!:])', r' \1', text)
    
    text = re.sub(r'([,()?!:])(?!\s)', r'\1 ', text)
    
    text = re.sub(r'\s{2,}', ' ', text)
    
    return text

In [ ]:
def change_orig_text_to_gend_swapped(txt_file, list_of_manually_changed_files):
    # create a new files with the old name but new text with swapped sentences. also add old ann file
    file_and_text_dict = {}
    file_need_to_be_changed = None

    ann_file = txt_file.replace(".txt", ".ann")
    changed_ann_file = ann_file.replace('ng', "ng_changed")
    changed_ann_file= changed_ann_file.replace('bruk', "bruk_changed")
    shutil.copyfile(ann_file, changed_ann_file)

    with open(txt_file, "r", encoding="utf-8") as txt_f:
        text = txt_f.read()

        new_text = []

        for x in text.split("\n"):
            if x:
                if x in orig_and_changed_dict.keys():
                    new_text.append(orig_and_changed_dict[x])
                    file_need_to_be_changed = txt_file
                else:
                    new_text.append(x)
            else:
                new_text.append("")

    
    changed_txt_file = txt_file.replace('ng', "ng_changed")
    changed_txt_file = changed_txt_file.replace('bruk', "bruk_changed")
    if new_text:
        if txt_file not in list_of_manually_changed_files:
            joined_text = "\n".join(new_text)
            file_and_text_dict[changed_txt_file] = joined_text
        else:
            with open(changed_txt_file, 'r') as file:
                changed_text = file.read()
            file_and_text_dict[changed_txt_file] = changed_text    
            print("add:", changed_txt_file)    

    return file_and_text_dict, file_need_to_be_changed

In [350]:
def dict_from_ann_files(directory):
    ann_files = find_ann_files(directory)
    ann_files_content = {}

    for ann_file in ann_files:
        with open(ann_file, "r", encoding="utf-8") as ann_f:
            rows = []
            buffer = []
            for line in ann_f:
                line = line.strip()
                if "\t" in line:
                    if buffer:
                        rows.append(parse_annotation_line("\n".join(buffer)))
                        buffer = []
                    buffer.append(line)
                else:
                    buffer.append(line)
            
            if buffer:
                rows.append(parse_annotation_line("\n".join(buffer)))

        ann_files_content[ann_file] = rows

    return ann_files_content

def parse_annotation_line(line):
    """
    Parses a single annotation line, including multi-line entities.
    """
    try:
        parts = line.split("\t", maxsplit=4)
        ind, label, start, end = parts[:4]
        ent = parts[4] if len(parts) > 4 else ""
        return {"ind": ind, "label": label, "start": int(start), "end": int(end), "ent": ent.replace("\n", " ")}
    except Exception as e:
        print(f"Error parsing annotation line: {line}")
        return None


In [390]:
from difflib import SequenceMatcher

def tokenize_with_indices(text):
    """Tokenizes text and returns a list of (token, start, end)."""
    doc = nlp(text)
    return [(token.text, token.idx, token.idx + len(token)) for token in doc]

def create_new_ann_file(orig_file_name, changed_file_name, changed_text, ann_files):
    with open(orig_file_name, 'r') as file:
        original_text = file.read()

    old_ann_file = orig_file_name.replace(".txt", ".ann")
    original_annotations = ann_files[old_ann_file]

    original_tokens = tokenize_with_indices(original_text)
    changed_tokens = tokenize_with_indices(changed_text)

    dict_for_changed_ann = {}
    index_offset_map = {}
    current_offset = 0
    orig_char_index = 0
    changed_char_index = 0

    orig_token_index = 0
    changed_token_index = 0

    while orig_char_index < len(original_text) and changed_char_index < len(changed_text):
        index_offset_map[orig_char_index] = current_offset

        if orig_token_index < len(original_tokens) and changed_token_index < len(changed_tokens):
            orig_token = original_tokens[orig_token_index]
            changed_token = changed_tokens[changed_token_index]

            if orig_char_index == orig_token[1] and changed_char_index == changed_token[1]:
                if orig_token[0] != changed_token[0]:
                    print("offest changed for: ", changed_token[0], orig_token[0])
                    current_offset += len(changed_token[0]) - len(orig_token[0])

                orig_char_index += len(orig_token[0])
                changed_char_index += len(changed_token[0])

                if orig_char_index == orig_token[2]:
                    orig_token_index += 1
                if changed_char_index == changed_token[2]:
                    changed_token_index += 1
            else:
                orig_char_index += 1
                changed_char_index += 1
        else:
            index_offset_map[orig_char_index] = current_offset
            orig_char_index += 1

    while orig_char_index < len(original_text):
        index_offset_map[orig_char_index] = current_offset
        orig_char_index += 1

    for annotation in original_annotations:
        orig_start = annotation['start']
        orig_end = annotation['end']
        new_start = orig_start + index_offset_map.get(orig_start, 0)
        new_end = orig_end + index_offset_map.get(orig_end, 0)

        new_ent = changed_text[new_start:new_end]
        dict_for_changed_ann[annotation['ind']] = (new_ent, (new_start, new_end))

    return dict_for_changed_ann

In [ ]:
ann_files = dict_from_ann_files("/Users/linndfors/study/diploma/ner_for_fem/data/v2.0/data/ng")

In [ ]:
new_ann_dict = {}

files_need_to_be_changed = set()

for old_file in tqdm(ng_files): # bruk_files
    ann_file = old_file.replace(".txt", ".ann")
    # ann_file = ann_file.replace("ng", "ng_changed")
    new_file_new_dev_dict,          file_need_to_be_changed = change_orig_text_to_gend_swapped(old_file)

    if file_need_to_be_changed:
        files_need_to_be_changed.add(file_need_to_be_changed)

    for file_path, content in new_file_new_dev_dict.items():
        os.makedirs(os.path.dirname(file_path), exist_ok=True)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(content)

        dict_for_changed_ann = {}
        if old_file in files_need_to_be_changed:
            dict_for_changed_ann = create_new_ann_file(old_file, file_path, content, ann_files_1)

        for x in ann_files_1[ann_file]:
            if x['ind'] in dict_for_changed_ann.keys():
                x['end'] = dict_for_changed_ann[x['ind']][1][1]
                x['start'] = dict_for_changed_ann[x['ind']][1][0]
                new_ent = dict_for_changed_ann[x['ind']][0]
                x['ent'] = new_ent


In [367]:
def write_ann_files_from_dict(ann_files_content):
    for ann_file, annotations in tqdm(ann_files_content.items()):
        relevant_txt_file = ann_file.replace(".ann", ".txt")
        if relevant_txt_file in files_need_to_be_changed:
            # ann_file = ann_file.replace("bruk/", "bruk_changed/")
            ann_file = ann_file.replace("ng/", "ng_changed/")
            with open(ann_file, "w", encoding="utf-8") as f:
                for annotation in annotations:
                    line = f"{annotation['ind']}\t{annotation['label']}\t{annotation['start']}\t{annotation['end']}\t{annotation['ent']}\n"
                    f.write(line)

write_ann_files_from_dict(ann_files)

100%|██████████| 298/298 [00:00<00:00, 1221.33it/s]
